# Get Voices

In [1]:
from pyneuphonic import Neuphonic, TTSConfig, Agent
from pyneuphonic.player import AudioPlayer

api_key = "172c4d78863aec707d4496f0d5258bff3389e92b2614c8175e7f18575eebc4a5.f13b6de5-100a-4252-b2fd-7a4aa39c852f" # GET THIS FROM beta.neuphonic.com!!!!!!!!!

In [3]:
client = Neuphonic(api_key=api_key)
voices = client.voices.get()  # get's all available voices

for voice in voices:
    print(voice)

id='e564ba7e-aa8d-46a2-96a8-8dffedade48f' name='Annie' tags=['Female', 'Twenties', 'American', 'Relaxing'] model_availability=['neu_hq']
id='8e9c4bc8-3979-48ab-8626-df53befc2090' name='Holly' tags=['Female', 'American', 'Midwestern'] model_availability=['neu_fast', 'neu_hq']
id='6ffede2d-d96e-4a9b-9d4d-e654b8ef4cf2' name='James' tags=['Male', 'Thirties', 'American'] model_availability=['neu_hq']
id='06fde793-8929-45f6-8a87-643196d376e4' name='Jo' tags=['Female', 'Thirties', 'American'] model_availability=['neu_hq']
id='ebf2c88e-e69d-4eeb-9b9b-9f3a648787a5' name='Marcus' tags=['Male', 'American'] model_availability=['neu_fast']
id='b19687fd-c5c9-4bda-9d52-756c3b10c88e' name='Miles' tags=['Male', 'American', 'Fourties', 'Narrator'] model_availability=['neu_hq']
id='f2185de7-e09b-46d7-9b20-8c82ef90524f' name='Wyatt' tags=['Male', 'American'] model_availability=['neu_hq']


# Text To Speech

In [3]:
client = Neuphonic(api_key=api_key)
sse = client.tts.SSEClient()
tts_config = TTSConfig(speed=1.05,  voice='8e9c4bc8-3979-48ab-8626-df53befc2090', model="neu_hq")

with AudioPlayer() as player:
    response = sse.send("""Neuphonic generates high quality and low latency text to speech. Experience the speed and clarity of our dynamically powered voice synthesis by entering your text!""", tts_config=tts_config)
    player.play(response)
    # player.save_audio('output.wav') 

# Altering Speech Parameters

In [ ]:
client = Neuphonic(api_key=api_key)
sse = client.tts.SSEClient()

# Below we alter the speed of the model and the voice
tts_config = TTSConfig(
    speed=1.3,  
    voice='b19687fd-c5c9-4bda-9d52-756c3b10c88e', 
    model="neu_hq"
)

with AudioPlayer() as player:
    response = sse.send("""Neuphonic generates high quality and low latency text to speech. Experience the speed and clarity of our dynamically powered voice synthesis by entering your text!""", tts_config=tts_config)
    player.play(response)
    # player.save_audio('output.wav') 

# Speech to Speech

Run the example below to create an agent that will greet you and initiate a voice conversation. 
Speak back to it to initiate a response.

In [ ]:
client = Neuphonic(api_key=api_key)

agent_id = client.agents.create(
    name='Agent 1',
    prompt='You are a helpful agent. Answer in 10 words or less.',
    greeting='Hi, how can I help you today?'
)['data']['id']

agent = Agent(client, agent_id=agent_id, tts_model='neu_hq')

await agent.start()

User: Does Jaffa cakes fit my diet?
Agent: It depends on your specific diet requirements and restrictions.
User: Can you
Agent: provide me with more details about your diet, such as your dietary restrictions, calorie goals, or specific diet plan?
User: me the nutritional information for Jaffa cakes?
Agent: Here is the approximate nutritional information for a standard Jaffa cake:
- Calories: 45
- Fat: 1g
- Carbohydrates: 8g
- Protein: 1g

Please note that nutritional values may vary slightly depending on the brand or recipe.
User: Is that correct? I don't know.
Agent: The provided nutritional information is an approximation and may vary.
User: Check it.
Agent: I apologize, but I'm not currently able to browse the internet and check the nutritional information for you. It's best to refer to the specific packaging or consult a reliable source for accurate information.
User: Yeah. You don't vaccinate it.
Agent: I'm sorry if I was unable to provide the specific information you were looking

CancelledError: 

User: Candidates 41.
Agent: I'm sorry, but I'm not sure what you mean by "Candidates 41." Can you please provide more context or clarify your question?
User: 45?
Agent: I apologize if there was a misunderstanding, but I'm unsure what "45" refers to in this context. Could you please provide more information or specify your question?


In [ ]:
import sounddevice as sd
import numpy as np
from scipy.io.wavfile import write

def record_until_silence(filename="recorded_audio.wav", sample_rate=44100, silence_threshold=0.01, silence_duration=1.0):
    print("Listening... (Speak to start recording)")
    
    duration_buffer = int(silence_duration * sample_rate)  # Buffer for silence detection
    audio_buffer = []
    silent_frames = 0

    def callback(indata, frames, time, status):
        nonlocal silent_frames, audio_buffer
        if status:
            print(status)

        # Check if audio is below silence threshold
        if np.max(np.abs(indata)) < silence_threshold:
            silent_frames += frames
        else:
            silent_frames = 0

        # Append audio to buffer
        audio_buffer.append(indata.copy())

        # Stop if silence exceeds threshold duration
        if silent_frames >= duration_buffer:
            raise sd.CallbackStop

    # Record audio in blocks
    try:
        with sd.InputStream(samplerate=sample_rate, channels=1, callback=callback):
            sd.sleep(60000)  # Arbitrary long timeout
    except sd.CallbackStop:
        print("Recording stopped.")

    # Combine and save recorded audio
    audio_data = np.concatenate(audio_buffer, axis=0)
    write(filename, sample_rate, (audio_data * 32767).astype(np.int16))  # Save as WAV
    print(f"Saved audio to {filename}")
    return filename

def main(api_url, api_key):
    # Step 1: Record audio until silence
    audio_file = record_until_silence()

    # Step 2: Send recorded audio to Neuphonic AI for transcription
    transcription_response = speech_to_text_neuphonic(audio_file, api_url, api_key)
    transcription = transcription_response.get("transcription", "")
    print("Transcription:", transcription)

    # Optional: Use transcription for web scraping or other analysis
    if transcription:
        results = scrape_web_for_query(transcription)
        print("Web Scraping Results:")
        for result in results:
            print(result)
    else:
        print("No transcription available.")

# Replace with your Neuphonic API details
api_url = "https://api.neuphonic.ai/speech-to-text"
api_key = "YOUR_API_KEY"

main(api_url, api_key)



FileNotFoundError: [Errno 2] No such file or directory: 'input.wav'

Note: To stop the above cell, you will need to either restart the Jupyter server or terminate the Jupyter server process.